Put all libraries here

In [13]:
import os, re, time, pickle
import pandas as pd, numpy as np, pickle
from chardet import detect

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Cleaning methods

In [4]:
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']

def detect_misspelling(source):
    pass

def replace_spelling(source):    
    return re.sub("Åf", "'", source)

Context Encoder

In [12]:
# referenced from DialogueGCN, mastodon code
def preprocess_text(x):
    for punct in '"!&?.,}-/<>#$%\()*+:;=?@[\\]^_`|\~':
        x = x.replace(punct, ' ')
    
    x = ' '.join(x.split())
    x = x.lower()
    
    return x


def load_pretrained_glove():
    print("Loading GloVe...")
    glv_vector = {}
    f = open('/embed/glove/glove.840B.300d.txt', encoding='utf-8')

    for line in f:
        values = line.split()
        word, coefs = values[0], np.asarray(values[1:], dtype='float')
        try:
            glv_vector[word] = coefs
        except ValueError:
            continue
    f.close()
    start_time = time.time()
    print(f"Took {time.time() - start_time} seconds to load pretrained GloVe model.")
    return glv_vector


Put main function here

In [3]:
os.listdir()

['.idea',
 '.ipynb_checkpoints',
 'data',
 'emotion_classifier.py',
 'prototype_context_encoder.ipynb']

In [5]:
print(get_encoding_type("data/train_sent_emo_dya.csv"))

MacRoman


In [6]:
X_train = pd.read_csv('data/train_sent_emo_dya.csv', encoding='MacRoman')

In [7]:
drop_features = list(X_train.keys()[6:])
drop_features.append("Emotion")
drop_features
y_train = pd.Series(X_train.Emotion)
X_train = X_train.drop(drop_features, axis=1)
X_train[:10]

,Utterance,Speaker,Sentiment,Dialogue_ID,Utterance_ID
0,also I was the point person on my companyÅfs t...,Chandler,neutral,0,0
1,You mustÅfve had your hands full.,The Interviewer,neutral,0,1
2,That I did. That I did.,Chandler,neutral,0,2
3,So letÅfs talk a little bit about your duties.,The Interviewer,neutral,0,3
4,My duties? All right.,Chandler,positive,0,4
5,"Now youÅfll be heading a whole division, so yo...",The Interviewer,neutral,0,5
6,I see.,Chandler,neutral,0,6
7,But thereÅfll be perhaps 30 people under you s...,The Interviewer,neutral,0,7
8,Good to know.,Chandler,neutral,0,8
9,We can go into detail,The Interviewer,neutral,0,9


In [8]:
y_train[:10]

0     neutral
1     neutral
2     neutral
3     neutral
4    surprise
5     neutral
6     neutral
7     neutral
8     neutral
9     neutral
Name: Emotion, dtype: object

In [9]:
X_train["Utterance"] = X_train["Utterance"].apply(lambda x: replace_spelling(x))
X_train[:10]

,Utterance,Speaker,Sentiment,Dialogue_ID,Utterance_ID
0,also I was the point person on my company's tr...,Chandler,neutral,0,0
1,You must've had your hands full.,The Interviewer,neutral,0,1
2,That I did. That I did.,Chandler,neutral,0,2
3,So let's talk a little bit about your duties.,The Interviewer,neutral,0,3
4,My duties? All right.,Chandler,positive,0,4
5,"Now you'll be heading a whole division, so you...",The Interviewer,neutral,0,5
6,I see.,Chandler,neutral,0,6
7,But there'll be perhaps 30 people under you so...,The Interviewer,neutral,0,7
8,Good to know.,Chandler,neutral,0,8
9,We can go into detail,The Interviewer,neutral,0,9


In [17]:
labels = set(y_train)
label_encoder = {label: i for i, label in enumerate(labels)}
label_decoder = {i: label for i, label in enumerate(labels)}

pickle.dump(label_encoder, open('data/dump/label_encoder.pkl', 'wb'))
pickle.dump(label_decoder, open('data/dump/label_decoder.pkl', 'wb'))